In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# 0. Notebook description

In this notebook, we load our cleaned dataset and perform a content-based recommendation based on the `Genre` column, which contains one or more movie genres.

# 1. Load dataset

In [11]:
movies_df = pd.read_csv('datasets/imdb_top_1000_cleaned.csv', low_memory=False)

movies_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,Star3,Star4,No_of_Votes
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142.0,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175.0,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152.0,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202.0,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96.0,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845


# 2. Prepare tf-idf model for the `Genre` column

The Genre column contains the genre(s) of each movie, which we used to build a content-based recommendation system. We transformed the genres into numeric vector representations using the tf-idf vectorizer.

Unlike traditional binary encoding, the tf-idf model accounts for the frequency of genres across the dataset, assigning higher importance to less common genres. This helps the system capture more nuanced relationships between movies based on their unique genre combinations. Each row in the resulting matrix represents a movie, and each column corresponds to a genre, with tf-idf scores indicating the movie's association with that genre.

This genre-based tf-idf matrix was used as input for the recommendation system, enabling it to recommend movies with similar genre profiles effectively.

In [12]:
# print the Genre column of the first 10 movies
print(movies_df['Genre'].head(10))

0                        Drama
1                 Crime, Drama
2         Action, Crime, Drama
3                 Crime, Drama
4                 Crime, Drama
5     Action, Adventure, Drama
6                 Crime, Drama
7    Biography, Drama, History
8     Action, Adventure, SciFi
9                        Drama
Name: Genre, dtype: object


In [13]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=2)

movies_df.Genre = movies_df.Genre.fillna('')
tfidf_model = vectorizer.fit_transform(movies_df.Genre)
print(f'Matrix contains {tfidf_model.shape[0]} movies and {tfidf_model.shape[1]} words')

Matrix contains 999 movies and 22 words


### Inspect the tf-idf model

Inspect the columns with popular movie genres like 'drama', 'fantasy', 'adventure', 'comedy', 'biography', 'crime' etc.

In [14]:
columns = vectorizer.get_feature_names_out()
print(columns)

['action' 'adventure' 'animation' 'biography' 'comedy' 'crime' 'drama'
 'family' 'fantasy' 'film' 'history' 'horror' 'music' 'musical' 'mystery'
 'noir' 'romance' 'scifi' 'sport' 'thriller' 'war' 'western']


In [15]:
genres = ['drama', 'fantasy', 'adventure', 'comedy', 'biography', 'crime']
columns = vectorizer.get_feature_names_out()
tfidf_model_df = pd.DataFrame.sparse.from_spmatrix(tfidf_model, columns=columns)
tfidf_model_df[genres].head()

,drama,fantasy,adventure,comedy,biography,crime
0,1.0,0,0,0,0,0
1,0.458335,0,0,0,0,0.888779
2,0.337001,0,0,0,0,0.653494
3,0.458335,0,0,0,0,0.888779
4,0.458335,0,0,0,0,0.888779


# 3. Find similar movies

To find similar movies, we use the KNN algorithm with **cosine similarity** as a distance metric to find the nearest neighbours.

In [16]:
def get_content_based_recommendation_genre(title, top_n=10, metric='cosine'):
    # Get the index of the movie that matches the title
    # we'll use that index to locate the row in the tf-idf matrix that corresponds to that movie
    idx = movies_df[movies_df.Series_Title.str.lower() == title.lower()].index[0]

    model = NearestNeighbors(n_neighbors=top_n+1, metric=metric)
    model.fit(tfidf_model)
    similar_movies = model.kneighbors(tfidf_model[idx], return_distance=False)[0]
    similar_movies = similar_movies[1:]  # remove the first item which is the movie itself

    # Return the top 10 most similar movies
    return movies_df.iloc[similar_movies]

In [17]:
get_content_based_recommendation_genre('The Godfather')[['Series_Title', 'Genre', 'IMDB_Rating', 'No_of_Votes', 'Genre']]

,Series_Title,Genre,IMDB_Rating,No_of_Votes,Genre
108,Scarface,"Crime, Drama",8.3,740911,"Crime, Drama"
707,À bout de souffle,"Crime, Drama",7.8,73251,"Crime, Drama"
1,The Godfather,"Crime, Drama",9.2,1620367,"Crime, Drama"
22,Cidade de Deus,"Crime, Drama",8.6,699256,"Crime, Drama"
111,Taxi Driver,"Crime, Drama",8.3,724636,"Crime, Drama"
762,This Is England,"Crime, Drama",7.7,115576,"Crime, Drama"
973,The Godfather: Part III,"Crime, Drama",7.6,359809,"Crime, Drama"
71,Once Upon a Time in America,"Crime, Drama",8.4,311365,"Crime, Drama"
396,Bound by Honor,"Crime, Drama",8.0,28825,"Crime, Drama"
894,Leviafan,"Crime, Drama",7.6,49397,"Crime, Drama"


In [18]:
get_content_based_recommendation_genre('The Dark Knight')[['Series_Title', 'Genre', 'IMDB_Rating', 'No_of_Votes', 'Genre']]

,Series_Title,Genre,IMDB_Rating,No_of_Votes,Genre
601,Ang-ma-reul bo-at-da,"Action, Crime, Drama",7.8,111252,"Action, Crime, Drama"
604,Ajeossi,"Action, Crime, Drama",7.8,62848,"Action, Crime, Drama"
895,Hell or High Water,"Action, Crime, Drama",7.6,204175,"Action, Crime, Drama"
781,Man on Fire,"Action, Crime, Drama",7.7,329592,"Action, Crime, Drama"
892,Sicario,"Action, Crime, Drama",7.6,371291,"Action, Crime, Drama"
344,Tropa de Elite 2: O Inimigo Agora é Outro,"Action, Crime, Drama",8.0,79200,"Action, Crime, Drama"
767,Lucky Number Slevin,"Action, Crime, Drama",7.7,299524,"Action, Crime, Drama"
967,Falling Down,"Action, Crime, Drama",7.6,171640,"Action, Crime, Drama"
849,Enter the Dragon,"Action, Crime, Drama",7.7,96561,"Action, Crime, Drama"
307,White Heat,"Action, Crime, Drama",8.1,29807,"Action, Crime, Drama"
